In [4]:
from datafin import FMPClient
from datafin.utils import (      #type: ignore
    format_date
)

from datafin_dagster.resources.constants import (
    FMP_API_KEY
)


import datetime
import os

In [6]:
fmp = FMPClient(
    api_key = FMP_API_KEY 
)

In [8]:
fmp.get_quote(
    'AMZN'
)

[{'symbol': 'AMZN',
  'name': 'Amazon.com, Inc.',
  'price': 216.98,
  'changePercentage': 1.59667,
  'change': 3.41,
  'volume': 38063980,
  'dayLow': 212.88,
  'dayHigh': 217.85,
  'yearHigh': 242.52,
  'yearLow': 151.61,
  'marketCap': 2303546472000,
  'priceAvg50': 192.684,
  'priceAvg200': 202.2147,
  'exchange': 'NASDAQ',
  'open': 214.76,
  'previousClose': 213.57,
  'timestamp': 1749499201}]

In [ ]:
import boto3
import json

client = boto3.client(
    'secretsmanager',
    aws_access_key_id = '',
    aws_secret_access_key = '',
    region_name = 'us-east-1'
)

In [19]:
c = client.get_secret_value(
    SecretId = 'apis/polygon'
)

In [20]:
json.loads(c['SecretString'])['polygon_api_key']

'GQn0EJI5jXsETWxkIn5fmVIQTufvfKam'